In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [ ]:
!pip install implicit

In [4]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - это товар, который купили пользователи, если они купиши товар из top5000. Используя этот товар мы смещяем качество рекомендаций. В какую сторону? Уберите этот товар и сравните с качеством на семинаре.

Качество уменьшается. Это дополнительная обобщающая информация, которую мы убираем, если не указываем.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [7]:
items = data_train['item_id'].value_counts()
items_logs = np.log(items)
items_weights = items_logs / items_logs.sum()

items_weights = items_weights.reset_index()
items_weights.columns = ['item_id', 'weights']

In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    weights = np.array(items_weights['weights'])
    #print(weights.sum())
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

Сделайте предсказания

In [9]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights))
result

Wall time: 12.1 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[880228, 1060143, 6395918, 1064115, 10356443]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[10311570, 6405523, 12524396, 93327, 5708755]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8124141, 878139, 95564, 7025088, 1056071]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[10150304, 1092251, 1807581, 7407761, 1371356]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1432031, 13213025, 985480, 12352405, 15925089]"
...,...,...,...
2037,2496,[6534178],"[5995692, 13039509, 10456110, 7134923, 969829]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[1078816, 925165, 237740, 849470, 6040007]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1595060, 939906, 14080414, 6633123, 925282]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[88505, 594269, 13158532, 829305, 1008154]"


## Считаем результаты моделей с вебинара

In [74]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [75]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 11.2 s


,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[880228, 1060143, 6395918, 1064115, 10356443]","[13512752, 5815004, 273331, 13416927, 2797746]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[10311570, 6405523, 12524396, 93327, 5708755]","[1153234, 9655203, 928774, 1002845, 825160]"


In [76]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [77]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 284 ms


,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[880228, 1060143, 6395918, 1064115, 10356443]","[13512752, 5815004, 273331, 13416927, 2797746]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[10311570, 6405523, 12524396, 93327, 5708755]","[1153234, 9655203, 928774, 1002845, 825160]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [99]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [100]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [101]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [103]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 1.97 s


In [104]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 225 ms


In [106]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 2.02 s


In [107]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 222 ms


In [109]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 1.89 s


In [110]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

Wall time: 205 ms


In [112]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 1.87 s


In [113]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

Wall time: 131 ms


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [48]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #p=rint(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    print(precision)
    
    
    return precision

In [115]:
def evaluete_pred(data, true):
    for col in data.columns[2:]:
        res = data.apply(lambda row: precision_at_k(row[col], row[true]), axis=1).mean()
        print(f'{col} : {res}')

In [116]:
evaluete_pred(result, 'actual')

weighted_random_recommendation : 0.00039177277179236047
random_recommendation : 0.0
popular_recommendation : 0.15523996082272082
itemitem : 0.13692458374142857
cosine : 0.13290891283055686
tfidf : 0.1389813907933383
own_purchases : 0.17969311132876015


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
-  *Попробуйте стратегии ансамблирования изученных алгоритмов

Обязательно нужно сделать первые 2 пункта!

Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров

In [126]:
data_test_top_5000 = data_test[data_test['item_id'].isin(top_5000)]
result_top5000 = data_test_top_5000.groupby('user_id')['item_id'].unique().reset_index()
result_top5000.columns=['user_id', 'actual']

In [127]:
items = data_train.item_id.unique()

result_top5000['random_recommendation'] = result_top5000['user_id'].apply(lambda x: random_recommendation(items, n=5))
result_top5000.head()

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[9396821, 847789, 872382, 882247, 1109370]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[825970, 853235, 952593, 1005184, 901747]"
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82...","[1094619, 9483663, 880423, 1065061, 927681]"
3,7,"[840386, 889774, 898068, 909714, 953476, 97699...","[819978, 1134174, 5585186, 982833, 1076056]"
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569...","[1011736, 879808, 851146, 967661, 6979803]"


In [128]:
%%time

popular_recs = popularity_recommendation(data_train, n=5)

result_top5000['popular_recommendation'] = result_top5000['user_id'].apply(lambda x: popular_recs)
result_top5000.head(2)

Wall time: 190 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[9396821, 847789, 872382, 882247, 1109370]","[999999, 6534178, 6533889, 1029743, 6534166]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[825970, 853235, 952593, 1005184, 901747]","[999999, 6534178, 6533889, 1029743, 6534166]"


In [129]:
result_top5000['weighted_random_recommendation'] = result_top5000['user_id'].apply(lambda x: weighted_random_recommendation(items_weights))
result_top5000.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[9396821, 847789, 872382, 882247, 1109370]","[999999, 6534178, 6533889, 1029743, 6534166]","[924691, 76246, 159814, 8299359, 13040152]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[825970, 853235, 952593, 1005184, 901747]","[999999, 6534178, 6533889, 1029743, 6534166]","[5565990, 954102, 12582395, 10254200, 840329]"
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82...","[1094619, 9483663, 880423, 1065061, 927681]","[999999, 6534178, 6533889, 1029743, 6534166]","[1133299, 1076206, 411844, 852420, 3617770]"
3,7,"[840386, 889774, 898068, 909714, 953476, 97699...","[819978, 1134174, 5585186, 982833, 1076056]","[999999, 6534178, 6533889, 1029743, 6534166]","[1059904, 5695324, 8159639, 970012, 5566109]"
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569...","[1011736, 879808, 851146, 967661, 6979803]","[999999, 6534178, 6533889, 1029743, 6534166]","[1003218, 879734, 2478148, 869916, 6379797]"


In [131]:
evaluete_pred(result_top5000, 'actual')

#popular ухудшился ?

random_recommendation : 0.0070921985815602774
popular_recommendation : 0.08855116514690954
weighted_random_recommendation : 0.0004052684903748734


Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [138]:
%%time

for k in range(1, 25):

    model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    results['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])
    print('k=',k,' res= ', results.apply(lambda row: precision_at_k(row['itemitem'], row['actual_x']), axis=1).mean())


k= 1  res=  0.1799869409076044



k= 2  res=  0.1920094678419821



k= 3  res=  0.1860920666013684



k= 4  res=  0.14495592556317158



k= 5  res=  0.13692458374142857


...


k= 20  res=  0.15259549461312247



k= 21  res=  0.15190989226248583



k= 22  res=  0.15083251714005697



k= 23  res=  0.15014691478942024



k= 24  res=  0.1500489715964723
Wall time: 1min 3s


### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.